In [ ]:
# Install or upgrade datasets and fsspec
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

In [ ]:
from datasets import load_dataset

ds = load_dataset("fancyzhx/amazon_polarity")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.81k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/260M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
# Let's work with a smaller subset to prevent memory issues
sample_size = 100000  # Adjust based on your system capabilities
train_df = ds['train'].shuffle(seed=42).select(range(sample_size)).to_pandas()
test_df = ds['test'].shuffle(seed=42).select(range(int(sample_size*0.2))).to_pandas()

# Explore the data
print(train_df.head())
print(f"\nTrain shape: {train_df.shape}, Test shape: {test_df.shape}")
print("\nLabel distribution in train set:")
print(train_df['label'].value_counts())

# Visualization

In [ ]:

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# 1. Distribution of Review Length
train_df['review_length'] = train_df['content'].apply(len)
plt.figure(figsize=(12, 6))
sns.histplot(train_df['review_length'], bins=50, kde=True)
plt.title('Distribution of Review Length (Characters)')
plt.xlabel('Number of Characters')
plt.ylabel('Frequency')
plt.axvline(train_df['review_length'].mean(), color='r', linestyle='--', label=f'Mean: {train_df["review_length"].mean():.0f} chars')
plt.legend()
plt.show()

# 2. Sentiment Class Distribution
plt.figure(figsize=(8, 5))
ax = sns.countplot(x='label', data=train_df, palette='viridis')
plt.title('Sentiment Class Distribution')
plt.xlabel('Sentiment Label')
plt.ylabel('Count')
plt.xticks([0, 1], ['Negative (0)', 'Positive (1)'])

# Add percentages
total = len(train_df)
for p in ax.patches:
    percentage = f'{100 * p.get_height()/total:.1f}%'
    x = p.get_x() + p.get_width()/2
    y = p.get_height() + 0.01*total
    ax.annotate(percentage, (x, y), ha='center')
plt.show()

# 3. Top 20 Most Frequent Words
def get_top_words(text_series, n=20):
    all_words = ' '.join(text_series).split()
    word_counts = Counter(all_words)
    return word_counts.most_common(n)

# Using processed text
text_source = train_df['processed_text'] if 'processed_text' in train_df.columns else train_df['content']
top_words = get_top_words(text_source)

plt.figure(figsize=(12, 6))
words, counts = zip(*top_words)
sns.barplot(x=list(counts), y=list(words), palette='viridis')
plt.title('Top 20 Most Frequent Words')
plt.xlabel('Frequency')
plt.ylabel('Word')
plt.show()

# 4. Word Clouds by Sentiment
def plot_wordcloud(text, title):
    wordcloud = WordCloud(width=800, height=400,
                         background_color='white',
                         colormap='viridis').generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, size=15)
    plt.axis('off')
    plt.show()

# Positive reviews
positive_text = ' '.join(train_df[train_df['label'] == 1]['content'])
plot_wordcloud(positive_text, 'Most Common Words in Positive Reviews')

# Negative reviews
negative_text = ' '.join(train_df[train_df['label'] == 0]['content'])
plot_wordcloud(negative_text, 'Most Common Words in Negative Reviews')

# Preprocessing


In [ ]:
# Text preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    # Remove extra whitespace
    text = re.sub(' +', ' ', text)
    # Tokenize
    words = text.split()
    # Remove stopwords and lemmatize
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

In [ ]:
# Apply preprocessing
train_df['processed_text'] = train_df['content'].apply(preprocess_text)
test_df['processed_text'] = test_df['content'].apply(preprocess_text)

# Check processed text
print("\nSample processed text:")
print(train_df['processed_text'].iloc[0])

# Tokenization and Sequence Preparation

In [ ]:
# Tokenization
max_words = 10000  # Vocabulary size
max_len = 200      # Maximum sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['processed_text'])

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_df['processed_text'])
X_test = tokenizer.texts_to_sequences(test_df['processed_text'])

# Pad sequences to ensure uniform length
X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

# Prepare labels
y_train = train_df['label'].values
y_test = test_df['label'].values

print("\nSample sequence:")
print(X_train[0])
print(f"\nSequence shapes - Train: {X_train.shape}, Test: {X_test.shape}")

# LSTM Model

In [ ]:
# Model parameters
embedding_dim = 128
lstm_units = 72
dropout_rate = 0.2

# Build the model
model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim),  # Removed input_length
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Dropout(dropout_rate),
    Bidirectional(LSTM(lstm_units)),
    Dropout(dropout_rate),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Explicitly build the model with input shape
model.build(input_shape=(None, max_len))  # This will properly initialize the model
model.summary()

In [ ]:
# Enhanced training parameters
batch_size = 128  # Increased to speed up training
epochs = 15  # Increased to allow more patience
validation_split = 0.2

# Enhanced early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Increased patience
    restore_best_weights=True,
    min_delta=0.001  # Minimum improvement required
)

# Add learning rate reduction
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=8,
    min_lr=1e-6,
    verbose=1
)

# Train the model
history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=[early_stopping, reduce_lr],  # Added reduce_lr
    verbose=1
)

In [ ]:
# Precision, Recall, and F1 Evaluation Cell
from sklearn.metrics import precision_recall_curve, f1_score, average_precision_score, precision_score, recall_score, classification_report
import matplotlib.pyplot as plt # Ensure plt is imported for plotting
import numpy as np # Ensure numpy is imported for array operations

def evaluate_prf1(model, X_test, y_test):
    # Generate predictions
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int)

    # Calculate metrics
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    ap = average_precision_score(y_test, y_pred_prob)  # Average precision

    # Classification report with zero_division handled
    report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'], zero_division=1)

    # Print metrics
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Average Precision: {ap:.4f}")
    print("\nDetailed Classification Report:")
    print(report)

    # Precision-Recall Curve
    precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_prob)

    plt.figure(figsize=(12, 5))

    # Plot Precision-Recall curve
    plt.subplot(1, 2, 1)
    plt.plot(recall_curve, precision_curve, color='blue', lw=2, label=f'PR Curve (AP = {ap:.2f})')
    plt.fill_between(recall_curve, precision_curve, alpha=0.2, color='blue')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="upper right")
    plt.grid(True)

    # F1 Score Threshold Analysis
    thresholds = np.linspace(0, 1, 100)
    f1_scores = [f1_score(y_test, (y_pred_prob > t).astype(int)) for t in thresholds]

    plt.subplot(1, 2, 2)
    plt.plot(thresholds, f1_scores, color='green', lw=2)
    plt.axvline(x=0.5, color='red', linestyle='--', label='Default Threshold (0.5)')
    plt.xlabel('Decision Threshold')
    plt.ylabel('F1 Score')
    plt.title('F1 Score vs. Decision Threshold')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Return metrics
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'average_precision': ap,
        'classification_report': report,
        'precision_curve': precision_curve,
        'recall_curve': recall_curve,
        'f1_thresholds': thresholds,
        'f1_scores': f1_scores
    }

# Run evaluation
prf1_results = evaluate_prf1(model, X_test, y_test)

# Additional Interpretation
print("\nKey Insights:")
print("- Precision: Measures how many predicted positives are truly positive (Higher = fewer false positives)")
print("- Recall: Measures how many actual positives were caught (Higher = fewer false negatives)")
print("- F1: Harmonic mean of precision and recall (Best when both matter equally)")
print("- Average Precision: Summary of PR curve (Higher = better overall performance)")
print("- Optimal Threshold: Check where F1 peaks in right plot (may not be at 0.5)")

In [ ]:
# Prediction Cell with Probability Scores
def predict_sentiment(model, tokenizer, reviews, max_len=200):
    """Predict sentiment for new reviews with probability scores"""

    # Preprocess the new reviews
    processed_reviews = [preprocess_text(review) for review in reviews]

    # Convert text to sequences and pad them
    sequences = tokenizer.texts_to_sequences(processed_reviews)
    X_new = pad_sequences(sequences, maxlen=max_len, padding='post')

    # Get predictions and probabilities
    predictions = model.predict(X_new)
    probs = np.concatenate([1 - predictions, predictions], axis=1)  # [prob_negative, prob_positive]

    return [
        {
            "review": review,
            "prediction": "POSITIVE" if pred > 0.5 else "NEGATIVE",
            "confidence": max(prob[1], 1 - prob[1]),  # Highest probability between the two classes
            "prob_negative": float(prob[0]),  # Convert numpy float to Python float
            "prob_positive": float(prob[1]),
            "interpretation": "Strong positive" if prob[1] > 0.9 else
                             "Positive" if prob[1] > 0.7 else
                             "Neutral" if 0.4 <= prob[1] <= 0.6 else
                             "Negative" if prob[1] > 0.3 else
                             "Strong negative"
        }
        for review, pred, prob in zip(reviews, predictions, probs)
    ]

# Sample reviews to predict
new_reviews = [
    "This product is amazing! Worth every penny.",
    "Terrible quality, broke after 2 days.",
    "It's okay but not what I expected.",
    "Absolutely love it! Best purchase this year.",
    "Waste of money. Doesn't work as advertised."
]

# Make predictions
predictions = predict_sentiment(model, tokenizer, new_reviews)

# Display results
print("Sentiment Predictions with Confidence Scores:")
for i, pred in enumerate(predictions, 1):
    print(f"\nReview {i}: {pred['review']}")
    print(f"Prediction: {pred['prediction']} ({pred['interpretation']})")
    print(f"Confidence: {pred['confidence']:.2%}")
    print(f"P(Negative): {pred['prob_negative']:.4f}")
    print(f"P(Positive): {pred['prob_positive']:.4f}")

In [ ]:
# Controversial Cases Evaluation Cell
import pandas as pd

def predict_with_confidence(model, tokenizer, reviews, max_len=200):
    """Predict sentiment with confidence scores for DataFrame output"""
    processed_reviews = [preprocess_text(review) for review in reviews]
    sequences = tokenizer.texts_to_sequences(processed_reviews)
    X_new = pad_sequences(sequences, maxlen=max_len, padding='post')

    predictions = model.predict(X_new)
    probs = np.concatenate([1 - predictions, predictions], axis=1)

    return [
        {
            "review": review,
            "prediction": "NEGATIVE" if pred < 0.5 else "POSITIVE",
            "confidence": max(prob[0], prob[1]),
            "prob_negative": float(prob[0]),
            "prob_positive": float(prob[1])
        }
        for review, pred, prob in zip(reviews, predictions, probs)
    ]

# Test controversial cases
controversial = [
    "Not bad for the price",
    "I don't hate it but would never recommend",
    "Meh, it's fine",
    "It works but I expected better",
    "Good enough I guess"
]

# Generate predictions and display as DataFrame
controversial_predictions = predict_with_confidence(model, tokenizer, controversial)
print(pd.DataFrame(controversial_predictions))

# Add interpretation analysis
print("\nControversial Case Analysis:")
for idx, pred in enumerate(controversial_predictions, 1):
    print(f"\nCase {idx}: '{pred['review']}'")
    print(f"Model is {pred['confidence']:.1%} confident it's {pred['prediction']}")
    if 0.4 < pred['prob_positive'] < 0.6:
        print("→ This appears to be a truly ambiguous case")
    elif (pred['prediction'] == "POSITIVE" and pred['prob_negative'] > 0.3) or \
         (pred['prediction'] == "NEGATIVE" and pred['prob_positive'] > 0.3):
        print("→ Model shows uncertainty despite prediction")

In [ ]:
from IPython.display import display, Javascript

# Clear all output and problematic metadata
display(Javascript('''
    IPython.notebook.clear_all_output();
    IPython.notebook.metadata.widgets = [];
    IPython.notebook.save_notebook();
    setTimeout(function() { alert("Notebook cleaned. Please save again."); }, 1000);
'''))